# Introduction to the Function Calling

The most powerful feature of chat completion is the ability to call functions from the model. This allows you to create a chat bot that can interact with your existing code, making it possible to automate business processes, create code snippets, and more.

With Semantic Kernel, we simplify the process of using function calling by automatically describing your functions and their parameters to the model and then handling the back-and-forth communication between the model and your code.

Read more about it [here](https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/function-calling).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.23.0

### Setting Up Execution Settings

Using `FunctionChoiceBehavior.Auto()` will enable automatic function calling. There are also other options like `Required` or `None` which allow to control function calling behavior. More information about it can be found [here](https://learn.microsoft.com/en-gb/semantic-kernel/concepts/ai-services/chat-completion/function-calling/function-choice-behaviors?pivots=programming-language-csharp).

In [2]:
#pragma warning disable SKEXP0001

OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

### Providing plugins to the Kernel
Function calling needs an information about available plugins/functions. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "prompt_template_samples");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [4]:
var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";

Since we imported available plugins to Kernel and defined the ask, we can now invoke a prompt with all the provided information. 

We can run function calling with Kernel, if we are interested in result only.

In [5]:
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

Here are some date ideas expressed as poems for Valentine's Day:

1. **Romantic Picnic:**
   In the park on a blanket so fine,  
   Two lovebirds with treats and some wine.  
   But the ants joined the feast,  
   And the wind blew the yeast—  
   Now their bread's flying high like a sign!  

2. **Candlelit Dinner:**
   In a bistro so cozy and bright,  
   Two lovers sat down for the night.  
   But the candles, they swayed,  
   And the waiter, dismayed,  
   Served spaghetti with just candlelight!  

3. **Movie Marathon:**
   On Valentine's Day, quite the plan,  
   A movie marathon began.  
   With popcorn so pink,  
   And chocolates that wink,  
   Cupid's snacks in a big frying pan!  

4. **Scenic Hike:**
   On Valentine's Day, we took a hike,  
   Up a hill with a view we both like.  
   But I tripped on a root,  
   And lost my left boot,  
   So we laughed all the way to the bike.  

5. **Wine Tasting Tour:**
   On Valentine's Day, we set sail,  
   To a vineyard with love in 

But we can also run it with `IChatCompletionService` to have an access to `ChatHistory` object, which allows us to see which functions were called as part of a function calling process. Note that passing a Kernel as a parameter to `GetChatMessageContentAsync` method is required, since Kernel holds an information about available plugins.

In [6]:
using Microsoft.SemanticKernel.ChatCompletion;

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var chatHistory = new ChatHistory();

chatHistory.AddUserMessage(ask);

var chatCompletionResult = await chatCompletionService.GetChatMessageContentAsync(chatHistory, openAIPromptExecutionSettings, kernel);

Console.WriteLine($"Result: {chatCompletionResult}\n");
Console.WriteLine($"Chat history: {JsonSerializer.Serialize(chatHistory)}\n");

Result: Here are some date ideas turned into playful poems for Valentine's Day:

1. **Romantic Picnic in the Park:**

   In the park with a basket so grand,  
   Two lovers had everything planned.  
   But ants joined the feast,  
   And wouldn't be ceased,  
   Now they're dining with nature's own band!  
   
   The cheese was as fancy as gold,  
   The wine, a story to be told.

2. **Candlelit Dinner at Home:**

   In a kitchen, a chef with a plan,  
   Cooked up dishes that barely began,  
   The pasta was glue,  
   The soup turned to stew,  
   But the candlelight hid every flan!  
   
   With a wink and a smile, they dined,  
   Ignoring the food that malarkied. 

3. **Wine Tasting Event:**

   At a wine tasting event, we did meet,  
   With glasses and cheese, quite the treat.  
   We swirled and we sipped,  
   Then giggled and tripped,  
   And left with a grape-stained receipt! 🍷

4. **Scenic Hot Air Balloon Ride:**

   In a basket up high we did float,  
   With a pilot who 